In [1]:
#path for input
path = ''

In [2]:
#UI here

'''
choices for the techniques that would be run

1. model selection : CNN based, Attention based, both
2. input perturbation test: occlusion, noise

Final results: Metrics: significance and what they mean
'''


'\nchoices for the techniques that would be run\n\n1. model selection : CNN based, Attention based, both\n2. input perturbation test: occlusion, noise\n\nFinal results: Metrics: significance and what they mean\n'

In [3]:
!nvidia-smi

Tue Oct 10 19:38:15 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off | 00000000:B1:00.0 Off |                  N/A |
| 22%   24C    P8               7W / 250W |    725MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
#verification in captum

In [5]:
#gradcam with saliency maps in pytorch



In [6]:
#abid: add ui code here

'\nInput\n1. image \n2. perturbation technique (occlusion, yada yada) \n3. introspection method\n4. resulting heatmaps\n5. metrics on display\n'

In [8]:
import random
import torch
import torchvision
from torch.nn import functional as F
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
import cv2
import os
import tensorflow as tf
from tensorflow import keras
import scipy.stats as sci 

2023-10-10 19:38:24.111044: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 19:38:31.846246: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /scratch/juneja/python/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda-11.3/lib64:/scratch/juneja/python/lib:/usr/local/cuda-11.3/tensorRT-8.0/lib64:/usr/local/cuda-11.3/cudnn-8.2/lib64:/usr/local/cuda-11.3/nccl-2.9/lib64
2023-10-10 19:38:31.846603: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7

In [9]:
import torch

print(torch.__version__)


1.12.1+cu113


In [10]:
#set proxy

os.environ['HTTP_PROXY'] = 'http://fp.cs.ovgu.de:3210/'
os.environ['HTTPS_PROXY'] = 'http://fp.cs.ovgu.de:3210/'

In [11]:
%matplotlib inline

import numpy as np
import skimage.transform
import torch
import torch.nn as nn
import torch.nn.functional as F
# from ScoreCAM import cam

from PIL import Image
from matplotlib.pyplot import imshow
from torchvision import models, transforms
from torchvision.utils import save_image

from cam import CAM, GradCAM, GradCAMpp, SmoothGradCAMpp, ScoreCAM

from utils.visualize import visualize, reverse_normalize
from utils.imagenet_labels import label2idx, idx2label

from scipy.spatial.distance import cdist
from pyemd import emd


In [12]:
#guy working on data science: woot woot!
# okay

#choose an image path
path = ''

#checkbox methods you want to run
methods = "GRADCAM, SCORECAM, LAYERCAM "

#let it run all perturbation methods
#occulusion, noising, etc

#report the metrics in a table
#Descirbe what the metrics do and report them in a table.

In [13]:
# pip install nbdev


In [14]:
import nbdev

In [15]:
pip install jupyterlab-quarto

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [16]:
'''

Step1: make user upload an image via image upload wizard

Step2: make transformations to the image in the background

Step3: run techniques selected sequentially
#checkboxes : gradcam, scorecam, whatever

for technique 
if(technique === 'gradcam','layercam','scorecam')
gradcam = gradcam(image)

Step4: make user select perturbation techniques:
1. Import perturbation_methods
2. make the image go through perturbation methods
3. run the explaination techniques on the perturbed images.

#checkboxes : occlusion, noise levels

Step 5: give out meaningful metrics
import metrics.py
metrics(saliency_map1, saliency_map2)

#table


'''

"\n\nStep1: make user upload an image via image upload wizard\n\nStep2: make transformations to the image in the background\n\nStep3: run techniques selected sequentially\n#checkboxes : gradcam, scorecam, whatever\n\nfor technique \nif(technique === 'gradcam','layercam','scorecam')\ngradcam = gradcam(image)\n\nStep4: make user select perturbation techniques:\n1. Import perturbation_methods\n2. make the image go through perturbation methods\n3. run the explaination techniques on the perturbed images.\n\n#checkboxes : occlusion, noise levels\n\nStep 5: give out meaningful metrics\nimport metrics.py\nmetrics(saliency_map1, saliency_map2)\n\n#table\n\n\n"

In [17]:
from ipywidgets.widgets import FileUpload, Label, FloatProgress, FloatSlider, Button, RadioButtons, Checkbox, Dropdown, HTML, Image, ToggleButtons
from ipywidgets.widgets import Layout, HBox, VBox
from IPython.display import display
# import bqplot as bq
# import time
# import threading

FileUpload(
    accept='image/*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

FileUpload(value={}, accept='image/*', description='Upload')

In [18]:
#| default_exp techniques

In [19]:
    # Preprocessing
    def preprocessing(image):
        size = (300, 300)
        image = image.resize(size)
        #mean and std from ImageNet
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
           std=[0.229, 0.224, 0.225]
        )
        preprocess = transforms.Compose([
            transforms.ToTensor(),
            normalize
        ])
        tensor = preprocess(image)
        # reshape 4D tensor (N, C, H, W)
        tensor = tensor.unsqueeze(0)
        return tensor

In [20]:
# tensor = preprocessing(image)
# tensor.shape

In [21]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True).to(device)
model.eval()

Using cache found in /home/juneja/.cache/torch/hub/pytorch_vision_v0.10.0
/scratch/juneja/python/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/scratch/juneja/python/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [22]:
# gpu_reference_tensor = next(model.parameters())
# tensor = tensor.type_as(gpu_reference_tensor)
# target_layer = model.layer4[1].conv2
# wrapped_model = ScoreCAM(model, target_layer)
# cam, idx = wrapped_model(tensor)
# classes = open("/project/validating_attribution_techniques/shardul/classes.txt").read().splitlines()

In [23]:
# fig, ax = plt.subplots()
# print(idx)
# img = reverse_normalize(tensor)
# heatmap = visualize(img, cam)
# heatmap = np.transpose(heatmap.squeeze(), (1, 2, 0))
# ax.imshow(heatmap, cmap='turbo', alpha = 0.8)
# ax.set_title(f"Class: {classes[idx]}")
# fig.savefig(f"/project/validating_attribution_techniques/shardul/output/saliency/before_noise/{idx2label[idx]}.jpeg")

In [24]:
#| hide
from nbdev.export import nb_export
nb_export('saliency_verification_tests.ipynb') # arg1 : name of notebook, arg2 : location of the export relative to the current directory

In [25]:
#| default_exp perturbation_methods

In [26]:
#| export

#Creating the patch
PATCH_SIZE = 60
def apply_grey_patch(path, image, top_left_x, top_left_y, patch_size):
    patched_image = np.array(image, copy=True)
    patched_image[top_left_y:top_left_y + patch_size, top_left_x:top_left_x + patch_size, :] = 0
    img = keras.preprocessing.image.array_to_img(patched_image)
    print(path)
    img.save(path)
    print(np.shape(patched_image))
    return patched_image

def add_random_noise_to_image(image, saliency_map, perturbation_strength=0.05, saliency_threshold=0.5, num_iterations=1, perturbation_prob=0.1):
    """perturbation_strength: This parameter controls the strength of the perturbation applied to the image. 
    It determines the magnitude of the noise added to the image.
    
    saliency_threshold: This parameter determines the percentage of saliency values to consider as the top values.
    It is used to threshold the saliency map and generate a binary mask.
    
    num_iterations: This parameter specifies the number of iterations to perturb the image. In each iteration, 
    the perturbation is applied to the image based on the saliency mask.
    
    perturbation_prob: This parameter controls the probability of perturbing each pixel in the saliency mask.
    It determines the randomness of the perturbation."""
    
    # Move the image tensor to the device of the saliency_map tensor
    image = image.to(saliency_map.device)
    
    # Get the top n percent of saliency values
    saliency_values = saliency_map.view(-1)
    threshold = torch.kthvalue(saliency_values, int((1 - saliency_threshold) * saliency_values.size(0)) + 1).values
    
    # Threshold the saliency map and generate a mask
    mask = torch.gt(saliency_map, threshold).float()
    
    # Perturb the image for a given number of iterations only at a random subset of the mask where the value is 1
    perturbed_image = image.clone()
    for i in range(num_iterations):
        perturbation_mask = (mask == 1) * (torch.rand_like(mask) < perturbation_prob).float()
        noise = torch.randn_like(perturbed_image) * perturbation_strength
        perturbation_mask = perturbation_mask.to(perturbed_image.device)  # Move perturbation_mask to the same device as perturbed_image
        noise = noise.to(perturbed_image.device)  # Move noise to the same device as perturbed_image
        perturbed_image = perturbed_image + perturbation_mask * noise
        perturbed_image = torch.clamp(perturbed_image, 0, 1)
        
    return perturbed_image

In [27]:
#| hide
from nbdev.export import nb_export
nb_export('saliency_verification_tests.ipynb','./') # arg1 : name of notebook, arg2 : location of the export relative to the current directory

In [28]:
#| default_exp metrics

In [29]:
#| export

import numpy as np
from skimage import img_as_float
from skimage import exposure

def normalize(x, method='standard', axis=None):
    '''Normalizes the input with specified method.

    Returns
    -------
    res : numpy.ndarray
        Normalized array.
    '''
    # TODO: Prevent divided by zero if the map is flat
    x = np.array(x, copy=False)
    if axis is not None:
        y = np.rollaxis(x, axis).reshape([x.shape[axis], -1])
        shape = np.ones(len(x.shape))
        shape[axis] = x.shape[axis]
        if method == 'standard':
            res = (x - np.mean(y, axis=1).reshape(shape)) / np.std(y, axis=1).reshape(shape)
        elif method == 'range':
            res = (x - np.min(y, axis=1).reshape(shape)) / (np.max(y, axis=1) - np.min(y, axis=1)).reshape(shape)
        elif method == 'sum':
            res = x / np.float_(np.sum(y, axis=1).reshape(shape))
        else:
            raise ValueError('method not in {"standard", "range", "sum"}')
    else:
        if method == 'standard':
            res = (x - np.mean(x)) / np.std(x)
        elif method == 'range':
            res = (x - np.min(x)) / (np.max(x) - np.min(x))
        elif method == 'sum':
            res = x / float(np.sum(x))
        else:
            raise ValueError('method not in {"standard", "range", "sum"}')
    return res


def match_hist(image, cdf, bin_centers, nbins=256):
    '''
    References
    ----------
    [1] Matlab implementation histoMatch(MTX, N, X) by Simoncelli, 7/96.
    '''
    image = img_as_float(image)
    old_cdf, old_bin = exposure.cumulative_distribution(image, nbins) # Unlike [1], we didn't add small positive number to the histogram
    new_bin = np.interp(old_cdf, cdf, bin_centers)
    out = np.interp(image.ravel(), old_bin, new_bin)
    return out.reshape(image.shape)

def normalize_map(map):
    min_value = np.min(map)
    max_value = np.max(map)
    normalized_map = (map - min_value) / (max_value - min_value)
    return normalized_map

def SIM(saliency_map1, saliency_map2):
    ''' 
    saliency maps when viewed as distributions, histogram intersection
    (SIM=1 means the distributions are identical).
    '''
    map1 = np.array(saliency_map1, copy=False)
    map2 = np.array(saliency_map2, copy=False)
    if map1.shape != map2.shape:
        map1 = resize(map1, map2.shape, order=3, mode='nearest') # bi-cubic/nearest is what Matlab imresize() does by default
    # Normalize the two maps to have values between [0,1] and sum up to 1
    map1 = normalize(map1, method='range')
    map2 = normalize(map2, method='range')
    map1 = normalize(map1, method='sum')
    map2 = normalize(map2, method='sum')
    # Compute histogram intersection
    intersection = np.minimum(map1, map2)
    return np.sum(intersection)

def CC(saliency_map1, saliency_map2):
    '''
    Pearson's correlation coefficient between two different saliency maps
    (CC=-1 inverse linear co-relation for CC=0 for uncorrelated maps, CC=1 for perfect linear correlation).
    '''
    map1 = np.array(saliency_map1, copy=False)
    map2 = np.array(saliency_map2, copy=False)
    if map1.shape != map2.shape:
        map1 = resize(map1, map2.shape, order=3, mode='nearest') # bi-cubic/nearest is what Matlab imresize() does by default
    # Normalize the two maps to have zero mean and unit std
    map1 = normalize(map1, method='standard')
    map2 = normalize(map2, method='standard')
    # Compute correlation coefficient
    return np.corrcoef(map1.ravel(), map2.ravel())[0,1]

def NSS(saliency_map, fixation_map):
    '''
    Normalized scanpath saliency of a saliency map, defined as the mean value of normalized (i.e., standardized) saliency map at fixation locations.
    You can think of it as a z-score. (Larger value implies better performance.)

    Parameters
    ----------
    saliency_map : real-valued matrix
        If the two maps are different in shape, saliency_map will be resized to match fixation_map..
    fixation_map : binary matrix
        Human fixation map (1 for fixated location, 0 for elsewhere).
    Returns
    -------
    NSS : float, positive
    '''
    s_map = np.array(saliency_map, copy=False)
    f_map = np.array(fixation_map, copy=False) > 0.5
    if s_map.shape != f_map.shape:
        s_map = resize(s_map, f_map.shape)
    # Normalize saliency map to have zero mean and unit std
    s_map = normalize(s_map, method='standard')
    # Mean saliency value at fixation locations
    return np.mean(s_map[f_map])


def AUC_Judd(saliency_map, fixation_map, jitter=True):
    '''
    AUC stands for Area Under ROC Curve.
    
    True positive (tp) rate correspond to the ratio of saliency map values above threshold
    at fixation locations to the total number of fixation locations.
    
    False positive (fp) rate correspond to the ratio of saliency map values above threshold
    at all other locations to the total number of possible other locations (non-fixated image pixels).

    AUC=0.5 is chance level.
    '''
    
    saliency_map = np.array(saliency_map, copy=False)
    fixation_map = np.array(fixation_map, copy=False) > 0.5
    # If there are no fixation to predict, return NaN
    if not np.any(fixation_map):
        print('no fixation to predict')
        return np.nan
    # Make the saliency_map the size of the fixation_map
    if saliency_map.shape != fixation_map.shape:
        saliency_map = resize(saliency_map, fixation_map.shape, order=3, mode='nearest')
    # Jitter the saliency map slightly to disrupt ties of the same saliency value
    if jitter:
        saliency_map += random.rand(*saliency_map.shape) * 1e-7
    # Normalize saliency map to have values between [0,1]
    saliency_map = normalize(saliency_map, method='range')

    S = saliency_map.ravel()
    F = fixation_map.ravel()
    S_fix = S[F] # Saliency map values at fixation locations
    n_fix = len(S_fix)
    n_pixels = len(S)
    # Calculate AUC
    thresholds = sorted(S_fix, reverse=True)
    tp = np.zeros(len(thresholds)+2)
    fp = np.zeros(len(thresholds)+2)
    tp[0] = 0; tp[-1] = 1
    fp[0] = 0; fp[-1] = 1
    for k, thresh in enumerate(thresholds):
        above_th = np.sum(S >= thresh) # Total number of saliency map values above threshold
        tp[k+1] = (k + 1) / float(n_fix) # Ratio saliency map values at fixation locations above threshold
        fp[k+1] = (above_th - k - 1) / float(n_pixels - n_fix) # Ratio other saliency map values above threshold
    return np.trapz(tp, fp) # y, x


In [30]:
#| hide
from nbdev.export import nb_export
nb_export('saliency_verification_tests.ipynb', '/commons/api/') # arg1 : name of notebook, arg2 : location of the export relative to the current directory